In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [3]:
tf.__version__

'2.1.0'

In [4]:
query="""
SELECT
 weight_pounds,
 is_male,
 mother_age,
 plurality,
 gestation_weeks
FROM
 publicdata.samples.natality
WHERE year > 2000
LIMIT 20000
"""

df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,8.187968,False,44,1,38.0
1,6.481591,False,15,1,37.0
2,6.188376,True,14,1,40.0
3,8.000575,False,15,1,41.0
4,6.437498,True,44,1,37.0


In [5]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,19985.000000,20000.000000,20000.000000,19778.000000
mean,7.293696,27.407900,1.032600,38.707503
std,1.318461,6.203744,0.185038,2.523860
min,0.562179,13.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.588395,50.000000,4.000000,47.000000


In [7]:
df['is_male'].value_counts()

True     10183
False     9817
Name: is_male, dtype: int64

In [8]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [9]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [10]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
17667,1,35,1,39.0
1511,0,18,1,39.0
7606,1,25,1,43.0
9397,1,26,1,40.0
17492,1,35,1,36.0
